## Convolutional Neural Networks
CNNs are great for problems that require classification (and sometimes regression) from visual data. CNNs are useful because *they* find the features themselves as opposed to us determining the features to use. We just give the CNN a label and data that is somewhat visual and let it go.
The general flow for this notebook will be:
1. Explore tools from PyTorch that allow for the import/transformation of different types of visual data
2. Importing data image data from a clothing database
3. Design a Multi-Class NN model that will be used to classify the different types of clothing in the images
4. Examine the results and then create a CNN model and compare the performance to the MCNN
5. Save the weights from model with the best results so that it can be used elsewhere

In [ ]:
# Lets start by importing everything we will need

import torch
from torch import nn

import torchvision
from torchvision import datasets # contains pre-built datasets that can be used to test models
from torchvision.transforms import ToTensor # contains useful functions that can transfrom common images formats to tensors

import matplotlib.pyplot as plt
print(f'PyTorch Version: {torch.__version__}, Torchvision Version: {torchvision.__version__}')

In [ ]:
# Lets get our training and testing data. Turns out torchvision has many built-in datasets already.
# We will be using the FashionMNIST dataset for this classification problem. It's basically the fashion
# version of the original MNIST dataset that used number. There are 10 classes, but they're clothes, not numbers.

DATA_DIR = "../../data/"

# Many of the datasets in this module have the same arguments.
train_data = datasets.FashionMNIST(
    DATA_DIR,
    train=True,
    transform=ToTensor(),
    target_transform=None,
    download=True
)

test_data = datasets.FashionMNIST(
    DATA_DIR,
    train=False,
    transform=ToTensor(),
    target_transform=None,
    download=True
)

In [ ]:
# With the training data donwloaded, lets explore
print(type(train_data))

# Lets get the first image. label combo from the test data
image, label = test_data[0]
print(f'Label Type: {type(label)}, Label: {label}')
print(f'Image Type: {type(image)}, Image Shape: {image.shape}')



#### So it looks like the image is a 3D tensor and the label is just an integer giving the class. The 1 in the first dimension of the image shows that it's just a greyscale image. This implies that the values in the 28x28 2D tensor just represent the intensity of the the pixel. If this were a color image, there would be 3 "channels" representing the intensity of RGB respectively. 

In [ ]:
# The dataset object has useful properties that allow you to view different aspects of the dataset.
print(f'Training size: {len(train_data)}, Test size: {len(test_data)}, Test/Train Ratio: {len(test_data)/len(train_data):.2f}')
print(train_data.classes)

In [ ]:
# Now lets actually look at some of the images/labels in the test dataset
fig = plt.figure(figsize=(9,9))
plt_rows = 4
plt_cols = 4
torch.manual_seed(42)
for sub in range(1, plt_rows * plt_cols + 1):
    index = torch.randint(0, len(train_data) - 1, size=[1]).item() # Get a random number (in range of the train data)
    img, label = train_data[index] # Pull a random test sample and it's label from the training data
    fig.add_subplot(plt_rows, plt_cols, sub)
    plt.imshow(img.squeeze(), cmap='gray') # Show the image in the subplot. Squeezing out the first dimension, not needed for 2D greyscale
    plt.title(train_data.classes[label])
    plt.axis(False)

#### As seen above, we have quite a few different images corresponding to the different classes in the dataset! The next step is create a Dataloader object for input into the model we plan to create. This seems like an arbitrary step, why do we need to transform the data into some special object as opposed to just using it as it exists in tensor form? The main reason is due to __Batching__. In most real world scenarios, datasets will be huge. So far, we've been doing a form of training called __Batch Gradient Descent__ where *backprop is done on the entire training set for each epoch*. This is computationally possible with small datasets, but with larger datasets, this would be computationally expensive (thus actually expensive in time and cost). The method we'll be adopting is the use of mini-batching; grouping a very small subset of the entire training set to be used for one backprop step. Instead of the loss from the WHOLE dataset (Batch) or each individual sample (Stochastic) being used to change the weights in the model, the *average* loss of the mini-batch is used. This is effective in reducing the computational load and to prevent issues like overfitting. Moving our data to a Dataloader object allows for us to easily change the batch size and to allow for shuffling the data around for each epoch (epoch # = mini-batch #).

In [ ]:
# Create the dataloaders obj for both training and test
from torch.utils.data import DataLoader
BATCH_SIZE = 32
train_dloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_dloader = DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Now lets inpect the objects. Much of the logic to batch and match labels to input data is done for us in the Dataloader obj
print(f'Type: {type(train_dloader)}')
train_batch_img, train_batch_labels = next(iter(train_dloader))
print(f'Batch Img: {train_batch_img.shape}, Batch Labels: {train_batch_labels.shape}') # So the dloader is basically a list of all the batches and their corressponding labels

### Model Creation
With all the data import and transfomation done, we can now create a baseline (very simple) model. Will start with a basic 2 layer network, one hidden layer, with a flattening layer as the input layer.

In [ ]:
INPUT_DIMS = train_data[0][0].shape[0]*train_data[0][0].shape[1] # our images are 28*28 and will be flattened out before hitting the NN
HIDDEN_UNITS = 10
OUTPUT_DIMS = HIDDEN_UNITS

class FashionMNISTModel0(nn.Module):
    def __init__(self, in_shape: int, out_shape: int, hidden_units: int) -> None:
        super().__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=in_shape, out_features=hidden_units, bias=True),
            nn.Linear(in_features=hidden_units, out_features=out_shape, bias=True)
        )
    def forward(self, x):
        self.network(x)

fmnist_v0 = FashionMNISTModel0(in_shape=INPUT_DIMS,
                               out_shape=OUTPUT_DIMS,
                               hidden_units=HIDDEN_UNITS)
fmnist_v0